In [168]:
import pandas as pd

In [169]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [170]:
DATA_FOLDER = "data_sources/"

EVENTS_DATA_FILE = "convertcsv.csv"
NEW_EVENTS_DATA_FILE = "alarms_v2.csv"

In [171]:
df_alarms = pd.read_csv(f"{DATA_FOLDER}/{EVENTS_DATA_FILE}", sep=";")

In [172]:
# for day not for prev 24h
# df_alarms['start_day'] = pd.to_datetime(df_alarms['start'])
# df_alarms['start_day'] = df_alarms['start_day'].apply(lambda x: x.strftime('%Y-%m-%d 00:00:00'))
# alarms_by_region = df_alarms.groupby(['region_id', 'start_day'])

In [173]:
df_alarms['start'] = pd.to_datetime(df_alarms['start'])
df_alarms['end'] = pd.to_datetime(df_alarms['end'])

# counts = alarms_by_region.size().reset_index(name='alarms_count')

# df_alarms = pd.merge(df_alarms, counts, on=['region_id','start_day'], how='left')

# df_alarms = df_alarms.drop(["start_day"], axis=1)
alarms_by_region = df_alarms.groupby(['region_id'])

counts = alarms_by_region.apply(lambda x: x.set_index('start').rolling('24H')['region_id'].count()).reset_index(name='alarms_24h_count')

df_alarms = pd.merge(df_alarms, counts, on=['region_id','start'], how='left')

df_alarms["alarms_24h_count"] = df_alarms["alarms_24h_count"].astype(int)

In [174]:
df_alarms.head(5)

,id,region_id,region_title,region_city,all_region,start,end,clean_end,intersection_alarm_id,alarms_24h_count
0,2,3,Вінниччина,Вінниця,0,2022-02-25 22:55:42,2022-02-25 23:41:53,2022-02-25 23:41:53,NaN,1
1,4,12,Львівщина,Львів,0,2022-02-26 06:26:17,2022-02-26 07:15:28,2022-02-26 07:15:28,NaN,1
2,5,14,Одещина,Одеса,0,2022-02-26 07:16:58,2022-02-26 07:47:03,2022-02-26 07:47:03,NaN,1
3,6,6,Житомирщина,Житомир,0,2022-02-26 08:05:54,2022-02-26 09:36:36,2022-02-26 09:36:36,NaN,1
4,7,3,Вінниччина,Вінниця,0,2022-02-26 08:39:39,2022-02-26 10:42:41,2022-02-26 10:42:41,NaN,2


In [175]:
df_alarms['start_hour'] = pd.to_datetime(df_alarms['start']).dt.floor('H')
df_alarms['end_hour'] = pd.to_datetime(df_alarms['end']).dt.floor('H')

alarms_in_regions = df_alarms.groupby(['start_hour', 'end_hour'])

for drange, group in alarms_in_regions:
    regions_count = len(group['region_id'].unique())
    df_alarms.loc[(df_alarms['start_hour'] == drange[0]) & (df_alarms['end_hour'] == drange[1]), 'regions_count'] = regions_count

df_alarms = df_alarms.drop(["start_hour", "end_hour"], axis=1)
df_alarms["regions_count"] = df_alarms["regions_count"].astype(int)

In [176]:
df_alarms.head(5)

,id,region_id,region_title,region_city,all_region,start,end,clean_end,intersection_alarm_id,alarms_24h_count,regions_count
0,2,3,Вінниччина,Вінниця,0,2022-02-25 22:55:42,2022-02-25 23:41:53,2022-02-25 23:41:53,NaN,1,1
1,4,12,Львівщина,Львів,0,2022-02-26 06:26:17,2022-02-26 07:15:28,2022-02-26 07:15:28,NaN,1,1
2,5,14,Одещина,Одеса,0,2022-02-26 07:16:58,2022-02-26 07:47:03,2022-02-26 07:47:03,NaN,1,1
3,6,6,Житомирщина,Житомир,0,2022-02-26 08:05:54,2022-02-26 09:36:36,2022-02-26 09:36:36,NaN,1,1
4,7,3,Вінниччина,Вінниця,0,2022-02-26 08:39:39,2022-02-26 10:42:41,2022-02-26 10:42:41,NaN,2,1


In [178]:
df_alarms.to_csv(f"{DATA_FOLDER}/{NEW_EVENTS_DATA_FILE}", sep=";", index=False)